## Podstawy podstaw analizy danych EEG w MNE-Python

### Materiały do przejrzenia, dotyczące kroków przygotowania danych (ang. *preprocessing*):  

https://sccn.ucsd.edu/wiki/Makoto%27s_preprocessing_pipeline  

https://www.frontiersin.org/articles/10.3389/fnins.2018.00309/full

https://www.frontiersin.org/articles/10.3389/fnins.2013.00267/full

https://www.biorxiv.org/content/early/2017/12/28/240044

### Wybór środowiska graficznego:  
https://matplotlib.org/faq/usage_faq.html#what-is-a-backend  
(będzie to miało znaczenie dla sposobu wyświetlania wykresów - wybór poniżej pozwoli wyświetlić je w osobnym oknie, ale przez to będą interaktywne)

In [1]:
import matplotlib
matplotlib.use('Qt5Agg')

Oprócz tego importujemy biblioteki niezbędne do pracy z danymi:  
**numpy** do operacji matematycznych  
**pandas** do działania na bazach danych  
**maplotlib** do generowania wykresów  
**mne** do analizy EEG

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne

### Wczytywanie danych

Moduł IO w MNE służy do wczytywania danych MEEG pochodzących z różnych źródeł i różnych systemów danych. My zbieraliśmy dane do formatu **.bdf** ([BioSemi Data Format](https://www.biosemi.com/faq/file_format.htm)).  
Jest to jednak po prostu zmodyfikowana wersja pliku **.edf** (*European Data Format*), więc wczytywanie wszelkich pochodnych tego standardu odbywa się za pomoca jednej funkcji:

In [3]:
data = mne.io.read_raw_edf('./0101 oczy otwarte.bdf', preload=True, montage='biosemi64');

Extracting EDF parameters from ./0101 oczy otwarte.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
The following EEG sensors did not have a position specified in the selected montage: ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp']. Their position has been left untouched.
Reading 0 ... 1468415  =      0.000 ...   717.000 secs...


<ipython-input-3-5a2467d817f9>:1: RuntimeWarning: The following EEG sensors did not have a position specified in the selected montage: ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp']. Their position has been left untouched.
  data = mne.io.read_raw_edf('./0101 oczy otwarte.bdf', preload=True, montage='biosemi64');


### Podgląd surowych danych

Pod przyciskiem "Help" znajdziecie legendę skrótów klawiszowych, które pozwolą Wam manipulować wyświetlaniem danych.

In [4]:
data.plot();

In [8]:
data.info

<Info | 18 non-empty fields
    bads : list | 0 items
    buffer_size_sec : float | 1.0
    ch_names : list | Fp1, AF7, AF3, F1, F3, F5, F7, FT7, FC5, ...
    chs : list | 80 items (EEG: 79, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : bool | True
    dev_head_t : Transform | 3 items
    dig : list | 67 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 417.0 Hz
    meas_date : int | 1515433506
    nchan : int | 80
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 2048.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    experimenter : NoneType
    file_id : NoneType
    gantry_angle : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_name : NoneType
    subject_info 

In [36]:
print(data.ch_names)

['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2', 'EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp', 'STI 014']


In [17]:
data.plot_psd();

Effective window size : 1.000 (s)


### Wybór elektrody referencyjnej

W zasadzie dowolna elektroda (oraz dowolna ich kombinacja) może stać się punktem odniesienia dla sygnału z pozostałych.  
Poniżej wykorzystujemy bardzo popularną referencję uśrednioną - wirtualny kanał będący zwykłą średnią ze wszystkich elektrod na skalpie.

In [7]:
data.set_eeg_reference();

Applying average reference.
Applying a custom EEG reference.


In [16]:
data.plot();

Poniżej te same dane odniesione do średniej z dwóch elektrod na linii środkowej.

In [13]:
data.set_eeg_reference(ref_channels=['Fz'], projection=False);

Applying a custom EEG reference.


### Filtrowanie

Wykorzystamy dwa najbardziej typowe rodzaje filtrów - **górnoprzepustowy** (highpass, chcemy pozbyć się wszystkiego poniżej danej częstotliwości) oraz **dolnoprzepustowy** (lowpass, chcemy pozbyć się wszystkiego powyżej danej częstotliwości).

In [18]:
data.filter(1,None)
data.filter(None,40)

Setting up high-pass filter at 1 Hz
l_trans_bandwidth chosen to be 1.0 Hz
Filter length of 6759 samples (3.300 sec) selected
Setting up low-pass filter at 40 Hz
h_trans_bandwidth chosen to be 10.0 Hz
Filter length of 677 samples (0.331 sec) selected


<RawEDF  |  0101 oczy otwarte.bdf, n_channels x n_times : 64 x 1468416 (717.0 sec), ~717.2 MB, data loaded>

Teraz możemy zobaczyć efekt naszego filtrowania.

In [20]:
data.plot_psd();

Effective window size : 1.000 (s)


Żeby móc eksperymentować z różnymi typami filtrów będzie trzeba po każdej aplikacji cofnąć do poprzedniego stanu wczytując raz jeszcze dane.

### Korekcja artefaktów - ICA

Za pomocą **analizy składowych niezależnych** (ang. Independent Component Analysis) będziemy próbowali zrekonstruować źródła, które wygenerowały nasze dane EEG.  
Komponentów zawsze jest maksymalnie tyle, ile źródeł danych (w naszym przypadku elektrod).

In [21]:
from mne.preprocessing import ICA

Wczytujemy na świeżo dane, wybieramy tylko kanały EEG (w formie listy) i dość agresywnie filtrujemy (wyrzucając wszystko, co nie znajduje się pomiędzy 1 a 30 Hz).

In [22]:
data = mne.io.read_raw_edf('./0101 oczy otwarte.bdf', preload=True, montage='biosemi64')
data.pick_channels(['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 
                    'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 
                    'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2'])
data.filter(1,30,fir_design='firwin')

Extracting EDF parameters from ./0101 oczy otwarte.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
The following EEG sensors did not have a position specified in the selected montage: ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp']. Their position has been left untouched.
Reading 0 ... 1468415  =      0.000 ...   717.000 secs...


<ipython-input-22-09582c3c7874>:1: RuntimeWarning: The following EEG sensors did not have a position specified in the selected montage: ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp']. Their position has been left untouched.
  data = mne.io.read_raw_edf('./0101 oczy otwarte.bdf', preload=True, montage='biosemi64')


Setting up band-pass filter from 1 - 30 Hz
l_trans_bandwidth chosen to be 1.0 Hz
h_trans_bandwidth chosen to be 7.5 Hz
Filter length of 6759 samples (3.300 sec) selected


<RawEDF  |  0101 oczy otwarte.bdf, n_channels x n_times : 64 x 1468416 (717.0 sec), ~717.2 MB, data loaded>

Tworzymy obiekt ICA, który dopsowujemy do naszych danych.

In [25]:
ica_data = ICA(method='picard', random_state=0, max_iter=500)
ica_data.fit(data)

Fitting ICA to data using 64 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 64
Fitting ICA took 438.3s.


<ICA  |  raw data decomposition, fit (picard): 1468416 samples, 64 components, channels used: "eeg">

Teraz możemy je sobie wyświetlić w postaci siatki.  
Kliknięcie na komponent myszą spowoduje otwarcie jego szczegółów w nowym oknie (czasem trzeba chwilę cierpliwie na to poczekać).

In [30]:
ica_data.plot_components(range(20), inst=data)

<Figure size 750x700 with 20 Axes>

    using multitaper spectrum estimation with 7 DPSS windows


C:\Users\marcin\AppData\Local\conda\conda\envs\mne\lib\site-packages\mkl_fft\_numpy_fft.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.rfft_numpy(a, n=n, axis=axis)


In [26]:
ica_data.plot_properties(data)

    using multitaper spectrum estimation with 7 DPSS windows


C:\Users\marcin\AppData\Local\conda\conda\envs\mne\lib\site-packages\mkl_fft\_numpy_fft.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.rfft_numpy(a, n=n, axis=axis)


[<Figure size 700x600 with 5 Axes>,
 <Figure size 700x600 with 5 Axes>,
 <Figure size 700x600 with 5 Axes>,
 <Figure size 700x600 with 5 Axes>,
 <Figure size 700x600 with 5 Axes>]

Możemy też wyświetlić aktywność źródeł w formie szeregów czasowych (to czasem pomaga zrozumieć czym te niezależne komponenty są).

In [37]:
ica_data.get_sources(data).plot()

<Figure size 1280x722 with 4 Axes>

https://labeling.ucsd.edu/labelfeedback